In [2]:
import numpy as np
import itertools
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
import plotly.graph_objects as go
from ipywidgets import interact

from ripser import ripser
from persim import plot_diagrams

import warnings
warnings.filterwarnings("ignore")

# Functions:

In [3]:
def distance(x0, x1, y0, y1):
    return np.sqrt((x0-x1)**2+(y0-y1)**2)
    
def simplex_finder(xs, ys, parameter, ax):
    points = range(len(xs))
    distances = []
    triangles_x = [] ; triangles_y = []
    for i in range(len(xs)):
        distances.append([])
        for j in range(len(ys)):
            distances[-1].append(distance(xs[i],xs[j],ys[i],ys[j]))

    all_two_fold_combinations = tuple(itertools.combinations(points, 2))
    all_three_fold_combinations = tuple(itertools.combinations(points, 3))
    triangles_x = []
    triangles_y = []
    for i in range(len(all_three_fold_combinations)):
        three_points = tuple(itertools.combinations(all_three_fold_combinations[i], 2))
        condition = distances[three_points[0][0]][three_points[0][1]]<parameter*2 and distances[three_points[1][0]][three_points[1][1]]<parameter*2 and distances[three_points[2][0]][three_points[2][1]]<parameter*2
        if condition:
            triangles_x.append([]) ; triangles_y.append([])
            triangles_x[-1].extend([xs[all_three_fold_combinations[i][j]] for j in range(3)])
            triangles_y[-1].extend([ys[all_three_fold_combinations[i][j]] for j in range(3)])
    triangles_x = np.array(triangles_x) ; triangles_y = np.array(triangles_y)
    xf = triangles_x.flatten() ; yf = triangles_y.flatten()
    x_in_triangle = np.unique(triangles_x.flatten()) ; y_in_triangle = []
    for i in range(len(x_in_triangle)): y_in_triangle.append(yf[list(xf).index(x_in_triangle[i])])

    """ax:"""
    for i,j in all_two_fold_combinations: #"""drawing 2-simplices"""
        if distance(xs[i],xs[j],ys[i],ys[j]) < parameter*2:
            ax.plot([xs[i],xs[j]],[ys[i],ys[j]], 'c-')
    for i in range(len(triangles_x)): #"""drawing 3-simplices"""
        ax.fill(triangles_x[i], triangles_y[i], alpha=.3, color='b')
    for x,y in zip(xs, ys): ax.add_patch(plt.Circle((x, y), parameter, alpha=.3, color = 'r'))


In [4]:
max_xlim = max_ylim = 5
points = 10
xs = (np.random.random(points)-.5) * max_xlim*2 #[2, 2, -2, -2]#
ys = (np.random.random(points)-.5) * max_ylim*2 #[-2, 2, 2, -2]#

fig = go.FigureWidget()
@interact(parameter = (0,max_xlim,max_xlim/50))
def update(parameter=0):
    fig,ax = plt.subplots(1,3, figsize=(17,5)); 
    ax[0].set_xlim(-max_xlim,max_xlim); ax[0].set_ylim(-max_xlim,max_ylim); ax[0].grid()
    xy = np.array((xs, ys)).T
    dgms = ripser(xy, maxdim=1, thresh=parameter*2)['dgms'] #; print(dgms[0])
    dgms[0]/=2 ; dgms[1]/=2
    dgms[0][:,1][dgms[0][:,1]>=parameter]=parameter #; print(dgms[0])
    dgms[1][:,1][dgms[1][:,1]>=parameter]=parameter #; print(dgms[0])
    ax[0].plot(xs,ys,'co')
    ax[0].set_title('Point cloud') ; ax[0].set_xlabel(r'$x$') ; ax[0].set_ylabel(r'$y$')
    simplex_finder(xs, ys, parameter, ax[0])
    y_ticks_H0 = np.linspace(0.1, max_xlim/2-.1, len(dgms[0][:,0]))
    y_ticks_H1 = np.linspace(max_xlim/2, max_xlim-.1, len(dgms[1][:,0]))
    ax[1].hlines(y=y_ticks_H0, xmin=dgms[0][:,0], xmax=dgms[0][:,1], linestyle='-', colors='g', label=r'$H_0$')
    ax[1].hlines(y=y_ticks_H1, xmin=dgms[1][:,0], xmax=dgms[1][:,1], linestyle='--', colors='r', label=r'$H_1$')
    ax[1].set_xlim(0,max_xlim); ax[1].set_ylim(0,max_xlim); ax[1].set_yticks([]); ax[1].set_title('Persistence barcode') ; ax[1].set_xlabel('Parameter')
    ax[1].legend()
    plot_diagrams(dgms, ax=ax[2], xy_range=[0,max_xlim,0,max_ylim]) ; ax[2].set_title('Persistence diagram');

interactive(children=(FloatSlider(value=0.0, description='parameter', max=5.0), Output()), _dom_classes=('widg…